In [77]:
#!pip install missingno
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import missingno as msno
pd.set_option('display.max_rows', None)  # 모든 행을 출력
pd.set_option('display.max_columns', None)  # 모든 열을 출력
from sklearn.tree import DecisionTreeClassifier

In [67]:
train['Weight_in_gms'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 6999 entries, 0 to 6998
Series name: Weight_in_gms
Non-Null Count  Dtype
--------------  -----
6999 non-null   int64
dtypes: int64(1)
memory usage: 54.8 KB


In [71]:
train = pd.read_csv("train_fix_1.csv")
test = pd.read_csv("test_fix_1.csv")

# '?' 값을 랜덤한 문자열로 대체
train['Weight_in_gms'] = train['Weight_in_gms'].replace('?', str(np.random.randint(1000, 6001)))
test['Weight_in_gms'] = test['Weight_in_gms'].replace('?', str(np.random.randint(2000, 4001)))

# 결측치를 랜덤한 정수로 대체
train['Weight_in_gms'].fillna(np.random.randint(1000, 6001), inplace=True)
test['Weight_in_gms'].fillna(np.random.randint(2000, 4001), inplace=True)

# 데이터 유형을 정수로 변환
train['Weight_in_gms'] = train['Weight_in_gms'].astype('int64')
test['Weight_in_gms'] = test['Weight_in_gms'].astype('int64')

# 'Weight_in_gms' 열의 값을 100의 배수로 변경
#train['Weight_in_gms'] = ((train['Weight_in_gms'] - 1) // 100 + 1) * 100
#test['Weight_in_gms'] = ((test['Weight_in_gms'] - 1) // 100 + 1) * 100




# '?' 값을 NaN으로 대체
train['Discount_offered'].replace('?', np.nan, inplace=True)
test['Discount_offered'].replace('?', np.nan, inplace=True)

# 75% 분위수 계산
q75 = train['Discount_offered'].quantile(0.75)
q75 = test['Discount_offered'].quantile(0.75)

# '?' 값을 75% 분위수 값으로 변경
train['Discount_offered'].fillna(q75, inplace=True)
test['Discount_offered'].fillna(q75, inplace=True)

# 결측치를 75%는 10으로, 25%는 11로 채우기
q75 = train['Discount_offered'].quantile(0.75)
q25 = train['Discount_offered'].quantile(0.25)
q75 = test['Discount_offered'].quantile(0.75)
q25 = test['Discount_offered'].quantile(0.25)

train['Discount_offered'].fillna(11, limit=int(0.75 * len(train)), inplace=True)
train['Discount_offered'].fillna(7, limit=int(0.25 * len(train)), inplace=True)

test['Discount_offered'].fillna(11, limit=int(0.75 * len(train)), inplace=True)
test['Discount_offered'].fillna(7, limit=int(0.25 * len(train)), inplace=True)

train['Discount_offered'] = train['Discount_offered'].astype('int64')
test['Discount_offered'] = test['Discount_offered'].astype('int64')







import numpy as np

# 시드 설정
np.random.seed(0)

# 결측치 인덱스 추출
missing_indexes = train[train['Prior_purchases'].isna()].index
missing_indexes_1 = test[test['Prior_purchases'].isna()].index

# 결측치 개수 및 비율 계산
num_missing = len(missing_indexes)
num_missing_1 = len(missing_indexes_1)

num_3 = int(0.75 * num_missing)
num_4 = num_missing - num_3

num_3_1 = int(0.75 * num_missing_1)
num_4_1 = num_missing_1 - num_3_1

# 결측치를 채울 값 생성
values_to_fill = [3] * num_3 + [1] * num_4
values_to_fill_1 = [3] * num_3_1 + [4] * num_4_1

# 결측치 채우기
np.random.shuffle(values_to_fill)
np.random.shuffle(values_to_fill_1)

train.loc[missing_indexes, 'Prior_purchases'] = values_to_fill
test.loc[missing_indexes_1, 'Prior_purchases'] = values_to_fill_1





# Customer_care_calls  - 0 으로


train['Customer_care_calls'].fillna(0, inplace=True)  
test['Customer_care_calls'].fillna(0, inplace=True)  







train['Mode_of_Shipment'].replace('?', 'Ship', inplace=True)
test['Mode_of_Shipment'].replace('?', 'Ship', inplace=True)



# 중요도 바꿔보기
# 트레인은 하이 테스트도 하이
# 미디움 로우 = 
train['Product_importance'].replace('?', 'medium', inplace=True)
test['Product_importance'].replace('?', 'low', inplace=True)






In [72]:
# 원핫 인코딩
one_hot_train = pd.get_dummies(train)

one_hot_test = pd.get_dummies(test)


In [73]:
one_hot_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999 entries, 0 to 6998
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Customer_care_calls        6999 non-null   float64
 1   Customer_rating            6999 non-null   int64  
 2   Cost_of_the_Product        6999 non-null   int64  
 3   Prior_purchases            6999 non-null   float64
 4   Discount_offered           6999 non-null   int64  
 5   Weight_in_gms              6999 non-null   int64  
 6   Reached.on.Time_Y.N        6999 non-null   int64  
 7   Warehouse_block_A          6999 non-null   uint8  
 8   Warehouse_block_B          6999 non-null   uint8  
 9   Warehouse_block_C          6999 non-null   uint8  
 10  Warehouse_block_D          6999 non-null   uint8  
 11  Warehouse_block_F          6999 non-null   uint8  
 12  Mode_of_Shipment_Flight    6999 non-null   uint8  
 13  Mode_of_Shipment_Road      6999 non-null   uint8

In [74]:
one_hot_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Customer_care_calls        4000 non-null   float64
 1   Customer_rating            4000 non-null   int64  
 2   Cost_of_the_Product        4000 non-null   int64  
 3   Prior_purchases            4000 non-null   float64
 4   Discount_offered           4000 non-null   int64  
 5   Weight_in_gms              4000 non-null   int64  
 6   Warehouse_block_A          4000 non-null   uint8  
 7   Warehouse_block_B          4000 non-null   uint8  
 8   Warehouse_block_C          4000 non-null   uint8  
 9   Warehouse_block_D          4000 non-null   uint8  
 10  Warehouse_block_F          4000 non-null   uint8  
 11  Mode_of_Shipment_Flight    4000 non-null   uint8  
 12  Mode_of_Shipment_Road      4000 non-null   uint8  
 13  Mode_of_Shipment_Ship      4000 non-null   uint8

In [75]:
one_hot_train.to_csv('train_fix_09_14_2.csv', index=False)
one_hot_test.to_csv('test_fix_09_14_2.csv', index=False)

In [76]:
# 09_14 - 57


,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship,Product_importance_high,Product_importance_low,Product_importance_medium
Customer_care_calls,1.000000,0.009423,0.144350,0.080144,-0.055720,-0.102770,-0.061375,0.010856,-0.018672,0.003755,-0.001439,0.004297,0.013443,0.011019,-0.019065,-0.034266,0.030049,-0.011297
Customer_rating,0.009423,1.000000,0.006108,0.012742,-0.011650,0.007525,0.001127,-0.004241,0.002265,-0.000973,0.009332,-0.005045,-0.000894,-0.005619,0.005105,0.006696,-0.003362,-0.000318
Cost_of_the_Product,0.144350,0.006108,1.000000,0.111859,-0.096344,-0.128093,-0.068100,-0.023988,0.017234,0.025387,0.007850,-0.021040,-0.017730,0.020011,-0.001985,-0.033712,0.041745,-0.023373
Prior_purchases,0.080144,0.012742,0.111859,1.000000,-0.049044,-0.147039,-0.050858,0.001552,-0.001208,0.006031,-0.000079,-0.005023,0.010076,0.012705,-0.017781,0.010780,-0.015125,0.009262
Discount_offered,-0.055720,-0.011650,-0.096344,-0.049044,1.000000,-0.249663,0.276998,0.003458,-0.001116,-0.017903,0.015259,0.000309,-0.003290,0.003469,-0.000177,0.030130,-0.011970,-0.004609
Weight_in_gms,-0.102770,0.007525,-0.128093,-0.147039,-0.249663,1.000000,-0.259872,0.004576,-0.003727,0.000135,-0.012560,0.009164,-0.003274,-0.011131,0.011277,0.075945,-0.081108,0.039638
Reached.on.Time_Y.N,-0.061375,0.001127,-0.068100,-0.050858,0.276998,-0.259872,1.000000,-0.011003,0.008610,0.007706,0.018296,-0.018720,0.005365,0.003348,-0.006785,0.024041,-0.012308,-0.000904
Warehouse_block_A,0.010856,-0.004241,-0.023988,0.001552,0.003458,0.004576,-0.011003,1.000000,-0.200306,-0.202576,-0.200926,-0.315204,0.002744,0.001605,-0.003385,0.005182,0.014481,-0.017436
Warehouse_block_B,-0.018672,0.002265,0.017234,-0.001208,-0.001116,-0.003727,0.008610,-0.200306,1.000000,-0.201330,-0.199691,-0.313267,-0.007195,0.001446,0.004436,-0.013011,-0.017180,0.024480
Warehouse_block_C,0.003755,-0.000973,0.025387,0.006031,-0.017903,0.000135,0.007706,-0.202576,-0.201330,1.000000,-0.201953,-0.316816,-0.003143,0.004201,-0.000865,0.001247,0.002393,-0.003097
